<a href="https://www.kaggle.com/code/juancamilo0218/test-doom?scriptVersionId=285627809" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Esta celda instala las librerías necesarias para ejecutar ViZDoom en Kaggle. Incluye dependencias del sistema (Boost, SDL2, OpenAL, FFmpeg) y las versiones correctas de vizdoom, gym, pyvirtualdisplay, imageio y PyTorch para asegurar compatibilidad y permitir el entrenamiento y visualización del entorno.

In [1]:
!apt-get update -y && apt-get install -y libboost-all-dev cmake libsdl2-dev libopenal-dev ffmpeg
!pip install vizdoom[gym] gym==0.26.5 pyvirtualdisplay==3.0.0 imageio==2.27.0
!pip install torch torchvision

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]      
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]           
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,202 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,287 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,185 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64

En esta celda importamos todas las librerías necesarias para entrenar un agente de aprendizaje por refuerzo en VizDoom. Se incluyen herramientas para manejo del entorno (gym, imageio), procesamiento de imágenes (PIL), cálculo numérico (numpy) y construcción de redes neuronales (PyTorch).
Finalmente, se fija una semilla aleatoria (SEED = 42) para asegurar reproducibilidad en los resultados del entrenamiento.

In [2]:
import os
import gym
import time
import random
import numpy as np
from collections import deque
from PIL import Image
import imageio
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


Esta celda actualiza los repositorios de Ubuntu e instala todas las dependencias del sistema necesarias para que VizDoom pueda compilar y funcionar correctamente en Kaggle, incluyendo librerías gráficas, de sonido y herramientas de construcción como cmake y ffmpeg.

In [3]:
!apt-get update -y
!apt-get install -y \
    cmake \
    libboost-all-dev \
    libsdl2-dev \
    libopenal-dev \
    libjpeg-dev \
    libbz2-dev \
    zlib1g-dev \
    libfluidsynth-dev \
    libgme-dev \
    libopenal-dev \
    timidity \
    ffmpeg


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease               
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease                     
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading p

Esta celda instala todas las dependencias necesarias para compilar e instalar ViZDoom desde cero en Google Colab/Kaggle.
Incluye librerías del sistema (audio, video, compresión, SDL2) y herramientas como cmake para construir el motor Doom.

In [4]:
!git clone https://github.com/mwydmuch/ViZDoom.git
%cd ViZDoom
!git checkout master

# Compilar librería base
!cmake -DCMAKE_BUILD_TYPE=Release .
!make -j4

# Instalar módulo de Python
%cd python
!pip install .


Cloning into 'ViZDoom'...
remote: Enumerating objects: 20364, done.
remote: Counting objects: 100% (3000/3000), done.
remote: Compressing objects: 100% (1071/1071), done.
remote: Total 20364 (delta 2221), reused 1947 (delta 1928), pack-reused 17364 (from 3)
Receiving objects: 100% (20364/20364), 74.62 MiB | 28.92 MiB/s, done.
Resolving deltas: 100% (13051/13051), done.
/kaggle/working/ViZDoom
Already on 'master'
Your branch is up to date with 'origin/master'.
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Unix-like system detected, s

Esta celda crea una clase personalizada de entorno Gym para VizDoom. Configura el juego, procesa las imágenes, define las acciones permitidas, administra el frame skip y el frame stacking, y permite que el agente interactúe con Doom como si fuera un entorno estándar de Reinforcement Learning.

In [5]:
import gym
import numpy as np
from collections import deque
from PIL import Image

from vizdoom import DoomGame, Mode, ScreenFormat, ScreenResolution


class VizdoomGymEnv(gym.Env):
    def __init__(self, config_path, frame_skip=4, stack_frames=4, resolution=(320,240), grayscale=True):
        super().__init__()
        
        # ---------------------------
        # CONFIGURACIÓN DEL JUEGO
        # ---------------------------
        self.game = DoomGame()                # Crea instancia del motor de VizDoom
        self.game.load_config(config_path)    # Carga el archivo .cfg con la configuración del escenario
        self.game.set_window_visible(False)   # Oculta ventana para más velocidad (modo headless)
        self.game.set_sound_enabled(False)    # Desactiva sonido para ahorrar recursos
        self.game.set_mode(Mode.PLAYER)       # Modo jugador: el agente controla al personaje

        # Configuración del formato de pantalla
        self.game.set_screen_format(ScreenFormat.RGB24)       # Formato RGB de 8 bits por canal
        self.game.set_screen_resolution(ScreenResolution.RES_320X240)  # Resolución interna del juego

        # *** LÍNEA IMPORTANTE: INICIALIZA EL JUEGO ***
        self.game.init()

        # ---------------------------
        # PARÁMETROS DE LA ENVIRONMENT
        # ---------------------------
        self.frame_skip = frame_skip          # Número de frames que se omiten entre acciones (acelera la simulación)
        self.stack_frames = stack_frames      # Cuántos frames se apilan para dar contexto temporal al agente
        self.grayscale = grayscale            # Si las imágenes se convierten a grises o no
        self.resolution = resolution          # Resolución final (después de procesar la imagen)
        
        # ---------------------------
        # DEFINICIÓN DE ACCIONES
        # Cada acción es un vector de 8 botones (pueden ser distintos para tu escenario)
        # ---------------------------
        self.actions = [
            [1,0,0,0,0,0,0,0], # 0: avanzar
            [0,1,0,0,0,0,0,0], # 1: retroceder
            [0,0,1,0,0,0,0,0], # 2: girar izquierda
            [0,0,0,1,0,0,0,0], # 3: girar derecha
            [0,0,0,0,1,0,0,0], # 4: strafe izquierda
            [0,0,0,0,0,1,0,0], # 5: strafe derecha
            [0,0,0,0,0,0,1,0], # 6: saltar (si el escenario lo permite)
            [0,0,0,0,0,0,0,1], # 7: disparar
            [1,0,0,0,0,0,0,1], # 8: avanzar y disparar
            [0,0,0,1,0,0,0,1], # 9: girar derecha + disparar
        ]
        
        # Espacio de acción: un número entero que representa un índice del arreglo anterior
        self.action_space = gym.spaces.Discrete(len(self.actions))

        # ---------------------------
        # ESPACIO DE OBSERVACIÓN
        # Definimos forma de la imagen procesada
        # ---------------------------
        obs_shape = (resolution[1], resolution[0], 1 if grayscale else 3)
        
        self.observation_space = gym.spaces.Box(
            0, 255, obs_shape, dtype=np.uint8   # imágenes entre 0 y 255
        )

        # ---------------------------
        # FRAME STACK
        # Guarda los últimos N frames para que la red pueda "ver movimiento"
        # ---------------------------
        self._frame_stack = deque(maxlen=stack_frames)

    # ---------------------------------------------------------
    # reset(): reinicia el episodio
    # ---------------------------------------------------------
    def reset(self):
        self.game.new_episode()     # Comienza un nuevo episodio

        state = self.game.get_state()   # Obtiene el primer frame del nuevo episodio

        if state is None:
            # Si por algún error no hay frame, devolvemos pantalla negra
            frame = np.zeros((self.observation_space.shape), dtype=np.uint8)
        else:
            raw = state.screen_buffer      # Imagen cruda desde VizDoom (RGB)
            frame = self._process_frame(raw)  # Procesar imagen (resize, gray, ...)
        
        # Reinicia el frame stack
        self._frame_stack.clear()
        
        # Mete el mismo frame varias veces para llenar el stack inicial
        for _ in range(self.stack_frames):
            self._frame_stack.append(frame)

        # Devuelve los frames apilados (H, W, stack_frames)
        return np.stack(self._frame_stack, axis=2)

    # ---------------------------------------------------------
    # step(): ejecuta 1 acción en el entorno
    # ---------------------------------------------------------
    def step(self, action_idx):
        action = self.actions[action_idx]   # Obtiene vector de botones
        reward = 0
        done = False

        # Ejecuta la acción durante "frame_skip" frames
        for _ in range(self.frame_skip):
            reward += self.game.make_action(action)  # Realiza acción y acumula reward
            done = self.game.is_episode_finished()    # Check si el episodio terminó

            if done:
                break

        if done:
            # Si terminó, se devuelve imagen negra
            obs = np.zeros(self.observation_space.shape, dtype=np.uint8)
        else:
            raw = self.game.get_state().screen_buffer    # Obtiene frame actual
            frame = self._process_frame(raw)             # Procesarlo
            self._frame_stack.append(frame)              # Actualiza stack
            obs = np.stack(self._frame_stack, axis=2)    # Genera estado final

        return obs, reward, done, {}

    # ---------------------------------------------------------
    # _process_frame(): procesa imagen (grises, resize, normalización)
    # ---------------------------------------------------------
    def _process_frame(self, frame):
        img = Image.fromarray(frame)  # Convierte numpy array a PIL Image
        
        if self.grayscale:
            img = img.convert("L")    # Convertir a escala de grises

        img = img.resize(self.resolution)   # Redimensionar imagen
        arr = np.array(img, dtype=np.uint8) # Convertir de nuevo a numpy

        if self.grayscale:
            arr = arr[..., np.newaxis]      # Añadir canal extra (H, W, 1)

        return arr

    # ---------------------------------------------------------
    # render(): devuelve la imagen actual (normalmente para debugging)
    # ---------------------------------------------------------
    def render(self, mode="rgb_array"):
        if self.game.is_episode_finished():
            # Si el episodio terminó, mostrar pantalla negra
            return np.zeros(self.observation_space.shape, dtype=np.uint8)
        
        return self.game.get_state().screen_buffer   # Regresa frame sin procesar

    # ---------------------------------------------------------
    # close(): cerrar el entorno correctamente
    # ---------------------------------------------------------
    def close(self):
        self.game.close()


**PreprocessFrame**

Toma una imagen del entorno.

La convierte a escala de grises.

La redimensiona a 84×84 píxeles.

Devuelve la imagen lista para usar en la red neuronal.

**FrameStack**

Guarda las últimas k imágenes (frames).

En reset(), duplica el primer frame para llenar el stack inicial.

En append(), agrega el nuevo frame y devuelve un tensor con los últimos k frames apilados.

Esto permite que el agente vea movimiento, no solo imágenes sueltas.

In [6]:
class PreprocessFrame:
    def __init__(self, shape=(84,84)):
        # shape: resolución final deseada para cada frame (ancho, alto)
        self.shape = shape

    def __call__(self, obs):
        # obs llega como un array (H, W) o (H, W, 1)
        # .squeeze elimina dimensiones de tamaño 1 si existen
        img = Image.fromarray(obs.squeeze())  

        # Convertimos la imagen a escala de grises ("L")
        # y redimensionamos a la forma deseada
        img = img.convert("L").resize(self.shape)

        # Devolvemos la imagen como array de uint8
        return np.array(img, dtype=np.uint8)
        
class FrameStack:
    def __init__(self, k):
        # k = número de frames que se van a apilar
        self.k = k

        # deque mantiene los últimos k frames
        # maxlen=k hace que se borre automáticamente el más antiguo al meter uno nuevo
        self.frames = deque(maxlen=k)

    def reset(self, frame):
        # Vacía la cola y la rellena con el mismo frame k veces
        # Esto evita que el agente vea "basura" al comienzo
        for _ in range(self.k):
            self.frames.append(frame)

        # Devuelve un stack (H, W, k)
        return np.stack(self.frames, axis=2)

    def append(self, frame):
        # Añade un nuevo frame al stack
        self.frames.append(frame)

        # Devuelve la pila actualizada (H, W, k)
        return np.stack(self.frames, axis=2)


**Convoluciones (self.conv)**

Extraen características de las imágenes (bordes, formas, enemigos, etc.).
Son 3 capas conv + ReLU.

**Cálculo de tamaño (dummy)**

Se usa una imagen falsa de prueba para saber cuántas neuronas salen de las convoluciones.

**Capa fully-connected (self.fc)**

Reduce las características a un vector de 512 valores.

**policy_logits**

Produce la probabilidad de cada acción que el agente puede tomar.

value_head
Predice el "valor" del estado, usado por PPO para aprender.

**forward(x)**

Normaliza la imagen (0–255 → 0–1).

Extrae características.

Pasa por la red.

In [7]:
class CNNBase(nn.Module):
    def __init__(self, input_channels, n_actions):
        # Inicializa módulo de PyTorch
        super().__init__()

        # ----------------------------------------------------
        # BLOQUE CONVOLUCIONAL (EXTRACCIÓN DE CARACTERÍSTICAS)
        # Arquitectura clásica del paper de DeepMind (DQN 2015)
        # ----------------------------------------------------
        self.conv = nn.Sequential(
            nn.Conv2d(input_channels, 32, 8, stride=4),  # 1ra conv: reduce tamaño ~4x
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2),              # 2da conv: reduce ~2x
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, stride=1),              # 3ra conv: mantiene tamaño
            nn.ReLU(),
            nn.Flatten(),                                # Aplana para conectar con la red totalmente conectada
        )

        # ----------------------------------------------------
        # CÁLCULO AUTOMÁTICO DEL TAMAÑO DE SALIDA DE LA CNN
        # Esto evita tener que calcular manualmente el tamaño
        # ----------------------------------------------------
        with torch.no_grad():
            # Creamos una imagen dummy con (batch=1, canales, 84, 84)
            dummy = torch.zeros(1, input_channels, 84, 84)
            
            # Pasamos por la CNN para saber la dimensión final
            conv_out = self.conv(dummy).shape[1]

        # ----------------------------------------------------
        # BLOQUE FULLY CONNECTED
        # ----------------------------------------------------
        self.fc = nn.Sequential(
            nn.Linear(conv_out, 512),  # Capa completamente conectada
            nn.ReLU()
        )

        # ----------------------------------------------------
        # CABEZAS DE ACTOR Y CRITIC (PPO/A2C)
        # ----------------------------------------------------
        self.policy_logits = nn.Linear(512, n_actions)  # Actor → logits de probabilidad de acciones
        self.value_head = nn.Linear(512, 1)             # Critic → estima el valor del estado

    # --------------------------------------------------------
    # forward():
    #   x → imagen (N, C, 84, 84)
    #   salida → (logits, value)
    # --------------------------------------------------------
    def forward(self, x):
        # Normalizamos de 0–255 a 0–1
        x = x / 255.0

        # Extraemos características
        features = self.conv(x)

        # Pasamos por la fully connected
        features = self.fc(features)

        # Actor y Critic
        logits = self.policy_logits(features)                # (N, n_actions)
        value  = self.value_head(features).squeeze(-1)       # (N,)

        return logits, value


**RolloutBuffer**

Es una clase que guarda todas las experiencias que el agente recolecta durante un rollout de PPO:

states: estados observados

actions: acciones tomadas

logprobs: log-probabilidades de esas acciones

rewards: recompensas obtenidas

dones: si el episodio terminó

values: estimaciones del valor del estado hechas por la red

La función clear() simplemente reinicia el buffer.

**compute_gae()**

Calcula el GAE (Generalized Advantage Estimation), una técnica usada en PPO para estimar:

Ventajas (advantages) → qué tan buena fue una acción comparada con lo esperado.

Retornos (returns) → suma de recompensas futuras corregidas con el valor del estado.

Esto ayuda a que el entrenamiento sea más estable y eficiente, reduciendo el ruido en los gradientes.

In [8]:
class RolloutBuffer:
    def __init__(self):
        # Lista de estados observados (imágenes o vectores del entorno)
        self.states = []
        # Acciones tomadas
        self.actions = []
        # Log-probabilidades de esas acciones bajo la política actual
        self.logprobs = []
        # Recompensas recibidas
        self.rewards = []
        # Indicador de fin de episodio (1 si terminó, 0 si no)
        self.dones = []
        # Valores estimados por la red crítica
        self.values = []

    def clear(self):
        # Reinicia todas las listas del buffer
        self.__init__()


def compute_gae(rewards, values, dones, gamma=0.99, lam=0.95):
    # Lista donde guardaremos las ventajas finales
    advantages = []

    # Acumulador del GAE
    gae = 0

    # Se agrega un valor extra para facilitar el cálculo del bootstrap
    values = values + [0]

    # Recorremos los pasos hacia atrás (como indica GAE)
    for step in reversed(range(len(rewards))):
        # Delta = TD-error para este paso
        delta = (
            rewards[step]
            + gamma * values[step + 1] * (1 - dones[step])
            - values[step]
        )

        # Fórmula general del GAE
        gae = delta + gamma * lam * (1 - dones[step]) * gae

        # Insertamos al inicio para mantener el orden correcto
        advantages.insert(0, gae)

    # Los returns se calculan como: Return = Value + Advantage
    returns = [adv + val for adv, val in zip(advantages, values[:-1])]

    return advantages, returns


Este código implementa la parte central del algoritmo PPO (Proximal Policy Optimization), que entrena a la red neuronal usando los datos recolectados durante los episodios.

**¿Qué hace esta clase?**

Recibe la red (actor-critic) y los hiperparámetros del método.

Calcula la actualización de la política y del valor usando PPO.

Optimiza la red neuronal para que tome mejores decisiones.

**Partes importantes**

1. Constructor (__init__)

Inicializa:

La red y el optimizador (Adam).

El clipping de PPO (clip_eps).

Número de epochs, tamaño de batch.

Pesos de las pérdidas (valor y entropía).

El dispositivo (CPU o GPU).

En resumen: configura el entrenador.

2. Método update(buffer)

Actualiza la red usando los datos recolectados:

Convierte los datos del buffer a tensores:

estados

acciones

logprobs antiguos

valores y recompensas

Calcula ventajas y retornos usando GAE
(indica qué tan buenas fueron las acciones realmente).

**Normaliza ventajas**
→ esto mejora la estabilidad del entrenamiento.

Entrena por múltiple epochs y batches:

Calcula nueva política y valores.

Calcula el ratio entre prob nueva / prob vieja.

Aplica el clipping PPO para evitar cambios bruscos.

**Calcula:**

loss de política

loss de valor

entropía (exploración)

Backpropagation + gradient clipping
para evitar explosión de gradientes.

In [9]:
class PPO:
    def __init__(self, net, lr=2.5e-4, clip_eps=0.2, epochs=4,
                 batch_size=64, value_coef=0.5, ent_coef=0.01, device="cpu"):

        # Red neuronal que estima la política y el valor
        self.net = net.to(device)

        # Optimizador Adam para actualizar los parámetros de la red
        self.optimizer = optim.Adam(self.net.parameters(), lr=lr)

        # Parámetro de recorte de PPO (clip ε)
        self.clip_eps = clip_eps

        # Número de épocas para entrenar en cada update
        self.epochs = epochs

        # Tamaño de cada minibatch
        self.batch_size = batch_size

        # Peso del término de pérdida del valor (value loss)
        self.value_coef = value_coef

        # Peso del término de entropía (exploración)
        self.ent_coef = ent_coef

        # CPU o GPU
        self.device = device


    def update(self, buffer):

        # Convertir estados a tensor (stack porque vienen como lista)
        states = torch.tensor(np.stack(buffer.states), dtype=torch.float32).to(self.device)

        # Acciones tomadas por el agente en cada estado
        actions = torch.tensor(buffer.actions, dtype=torch.int64).to(self.device)

        # Log-probabilidades de las acciones bajo la política vieja
        old_logprobs = torch.tensor(buffer.logprobs, dtype=torch.float32).to(self.device)

        # Calcula ventajas A_t y retornos G_t usando GAE
        advantages, returns = compute_gae(buffer.rewards, buffer.values, buffer.dones)

        # Convertir a tensores
        advantages = torch.tensor(advantages, dtype=torch.float32).to(self.device)
        returns = torch.tensor(returns, dtype=torch.float32).to(self.device)

        # Normalización de las ventajas (estabiliza entrenamiento)
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

        # Número total de muestras
        dataset_size = len(states)


        for _ in range(self.epochs):
            # Índices aleatorios para mezclar datos (shuffling)
            idxs = np.arange(dataset_size)
            np.random.shuffle(idxs)

            for start in range(0, dataset_size, self.batch_size):
                # Tomar un minibatch por índices
                batch_idx = idxs[start:start + self.batch_size]

                # Reordenar a formato (B, C, H, W) para la red convolucional
                b_states = states[batch_idx].permute(0, 3, 1, 2)

                # Pasar por la red → logits de política y estimación de V(s)
                logits, vals = self.net(b_states)

                # Convertir logits en probabilidades
                probs = F.softmax(logits, dim=-1)

                # Distribución categórica para muestreo de acciones
                dist = torch.distributions.Categorical(probs)

                # Nuevas log-probabilidades bajo la política actual π_θ
                new_logprobs = dist.log_prob(actions[batch_idx])

                # ratio = π_θ(a|s) / π_old(a|s)
                ratio = torch.exp(new_logprobs - old_logprobs[batch_idx])

                # Primer término del surrogate: ratio * ventaja
                surr1 = ratio * advantages[batch_idx]

                # Segundo término recortado a [1−ε, 1+ε]
                surr2 = torch.clamp(ratio, 1 - self.clip_eps, 1 + self.clip_eps) * advantages[batch_idx]

                # PPO usa el mínimo de los dos → evita actualizaciones agresivas
                policy_loss = -torch.min(surr1, surr2).mean()

                # Pérdida del valor (MSE entre V(s) y retorno)
                value_loss = F.mse_loss(vals, returns[batch_idx])

                # Entropía de la política para fomentar exploración
                entropy = dist.entropy().mean()

                # Pérdida total: política + valor - entropía
                loss = policy_loss + self.value_coef * value_loss - self.ent_coef * entropy

                # Backprop
                self.optimizer.zero_grad()
                loss.backward()

                # Evita gradientes demasiado grandes (clipping de gradiente)
                nn.utils.clip_grad_norm_(self.net.parameters(), 0.5)

                # Actualizar parámetros
                self.optimizer.step()


La función entrena al agente PPO en Doom siguiendo estos pasos:

**Prepara todo**

Procesa las imágenes (84×84).

Apila 4 frames para dar memoria al agente.

Crea el buffer donde se guardan experiencias.

**Bucle principal de entrenamiento**

Mientras no se llegue al número total de timesteps:

Limpia el buffer.

Ejecuta acciones durante rollout_length pasos.

Para cada paso:

El agente predice la acción y el valor.

Se ejecuta la acción en Doom.

Se guarda estado, acción, recompensa, done y valor.

Si la partida termina, se reinicia el entorno.

**Actualiza el PPO**

Con los datos del buffer, el agente aprende (backpropagation).

**Guarda el modelo**

Cada ciclo guarda ppo_doom.pth.

**Imprime progreso**

Muestra timesteps, episodios y avisos de guardado.

In [10]:
def train(env, agent, ppo, total_timesteps=1_000_000, rollout_length=2048,
          device="cpu", save_path="ppo_doom.pth"):

    # Procesador de frames: convierte la imagen en escala de grises y cambia tamaño a (84,84)
    obs_proc = PreprocessFrame((84,84))

    # Stacker: acumula 4 frames para dar memoria temporal al agente
    frame_stack = FrameStack(4)

    # Buffer donde se guardan transiciones para un update de PPO
    buffer = RolloutBuffer()

    timestep = 0     # contador global de pasos
    episode = 0      # número total de episodios completados

    # Reiniciar el entorno
    obs = env.reset()

    # Procesar primer frame
    proc = obs_proc(obs)

    # Crear estado inicial (4 frames)
    state = frame_stack.reset(proc)


    # Loop principal del entrenamiento
    while timestep < total_timesteps:

        # Limpiar datos viejos del buffer antes de un nuevo rollout
        buffer.clear()

        # Recolectar un batch de transiciones
        for _ in range(rollout_length):

            # Convertir el estado a tensor → batch (1,C,H,W)
            st_tensor = torch.tensor(state, dtype=torch.float32)\
                            .unsqueeze(0).permute(0,3,1,2).to(device)

            # Pasar por la red → logits (política) y V(s)
            logits, value = agent(st_tensor)

            # Probabilidades de acción usando softmax
            probs = F.softmax(logits, dim=-1)

            # Distribución categórica (acciones discretas)
            dist = torch.distributions.Categorical(probs)

            # Seleccionar acción por muestreo estocástico
            action = dist.sample().item()

            # Log-probabilidad de la acción seleccionada (para PPO)
            logp = dist.log_prob(torch.tensor(action).to(device)).item()

            # Ejecutar la acción en el entorno
            next_obs, reward, done, info = env.step(action)

            # Preprocesar siguiente frame
            proc = obs_proc(next_obs)

            # Actualizar stack de frames (nuevo estado)
            next_state = frame_stack.append(proc)

            # Guardar la transición en el buffer
            buffer.states.append(state)
            buffer.actions.append(action)
            buffer.logprobs.append(logp)
            buffer.rewards.append(reward)
            buffer.dones.append(done)
            buffer.values.append(value.item())

            # Avanzar al nuevo estado
            state = next_state
            timestep += 1

            # Si el episodio terminó → reiniciar entorno
            if done:
                episode += 1
                obs = env.reset()
                proc = obs_proc(obs)
                state = frame_stack.reset(proc)

        # Actualizar la política con el rollout recolectado
        ppo.update(buffer)

        # Guardar los pesos del agente
        torch.save(agent.state_dict(), save_path)

        print(f"Timestep {timestep} — Episode {episode} — Guardado {save_path}")


In [11]:
import cv2
import torch
import numpy as np


# ============================================================
# MANEJO SEGURO DEL FRAME
# ============================================================
def get_frame(obs):
    """
    Extrae un frame válido desde la observación (obs).
    
    El entorno puede devolver:
    - un diccionario con la clave "screen"
    - un arreglo numpy directo
    Este wrapper unifica ambos casos.
    """
    if isinstance(obs, dict):
        # Si envía un diccionario, tomamos obs["screen"]
        return fix_frame_gray(obs["screen"])
    
    if isinstance(obs, np.ndarray):
        # Si ya es un arreglo numpy, se procesa igual
        return fix_frame_gray(obs)
    
    # Si no corresponde a ninguno, error
    raise ValueError("Obs no reconocida")


# ============================================================
# CONVERTIR GRIS -> BGR PARA VIDEO
# ============================================================

def gray_to_bgr(frame):
    """
    Convierte un frame de 1 canal (gris) a BGR.
    Esto es necesario porque:
    - VideoWriter de OpenCV requiere 3 canales.
    """
    if frame.ndim == 2:
        # Si el frame es [H,W], se convierte a [H,W,3]
        return cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    
    # Si ya tiene 3 canales, no se toca
    return frame


def to_bgr(frame):
    """
    Función redundante pero explícita:
    Recibe un frame en escala de grises y lo convierte a 3 canales.
    """
    return cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)


# ============================================================
# PREPROCESS PARA LA RED
# ============================================================

def preprocess(frame):
    """
    Preprocesa el frame para alimentarlo a la red neuronal:
    - Reescala a (84,84)
    - Normaliza dividiendo por 255
    - Convierte a tensor con shape (1, 84, 84)
    """
    # Reescalar al tamaño estándar de Atari/Doom
    frame = cv2.resize(frame, (84, 84))

    # Normalización a rango [0,1]
    frame = frame / 255.0

    # Convertir a tensor PyTorch y agregar dimensión batch
    return torch.tensor(frame, dtype=torch.float32).unsqueeze(0)


In [12]:
def fix_frame_gray(frame):
    """
    Normaliza cualquier tipo de frame a formato GRIS (H, W).

    Esta función arregla los casos más comunes:
    - Frames con dimensión adicional (H,W,4,1)
    - Frames con 4 canales (stack)
    - Frames con 1 canal (H,W,1)
    - Frames RGB (H,W,3)
    - Frames ya en gris (H,W)

    Devuelve siempre: np.uint8 con shape (H, W)
    """
    f = frame  # Alias para trabajar más cómodo

    # ------------------------------------------------------------
    # Caso: frame con 4 dimensiones y canal 1 al final
    # Ejemplo: (H, W, 4, 1) → queremos (H, W, 4)
    # Esto suele aparecer si los frames vienen con dims extra.
    # ------------------------------------------------------------
    if f.ndim == 4 and f.shape[-1] == 1:
        f = np.squeeze(f, axis=-1)  # Eliminamos esa última dimensión

    # ------------------------------------------------------------
    # Caso: frame con 4 canales (H, W, 4)
    # A veces el entorno entrega un "stack" de 4 frames juntos.
    # Aquí tomamos solo el PRIMER frame.
    # ------------------------------------------------------------
    if f.ndim == 3 and f.shape[2] == 4:
        f = f[:, :, 0]  # Usar solo el primer canal/frame

    # ------------------------------------------------------------
    # Caso: frame gris pero con canal explícito (H, W, 1)
    # Lo convertimos a (H, W)
    # ------------------------------------------------------------
    if f.ndim == 3 and f.shape[2] == 1:
        f = np.squeeze(f, axis=-1)

    # ------------------------------------------------------------
    # Caso: imagen RGB (H, W, 3)
    # Convertimos a GRIS usando OpenCV
    # ------------------------------------------------------------
    if f.ndim == 3 and f.shape[2] == 3:
        f = cv2.cvtColor(f, cv2.COLOR_BGR2GRAY)

    # ------------------------------------------------------------
    # Caso: imagen ya en escala de grises (H, W)
    # Este es el formato final deseado.
    # ------------------------------------------------------------
    if f.ndim == 2:
        return f.astype(np.uint8)  # Asegurar tipo uint8 para OpenCV y video

    # ------------------------------------------------------------
    # Si llega aquí: no fue posible convertir
    # ------------------------------------------------------------
    raise ValueError(f"Formato imposible de convertir: shape={frame.shape}")


In [13]:
def extract_frame(obs):
    # Caso 1: diccionario estilo VizDoomGym
    if isinstance(obs, dict):
        frame = obs.get("screen", None)
        if frame is None:
            raise ValueError("El diccionario no contiene 'screen'")
        return frame
    
    # Caso 2: tu entorno devuelve directamente la imagen (numpy)
    if isinstance(obs, np.ndarray):
        return obs

    # Caso 3: devuelve tupla (imagen, variables)
    if isinstance(obs, (list, tuple)):
        for item in obs:
            if isinstance(item, np.ndarray):
                return item
    
    raise ValueError("No pude extraer un frame válido de la observación")


evaluate_random prueba el entorno sin inteligencia, usando acciones aleatorias, para ver qué tan difícil es el escenario.

Reinicia el entorno por varios episodios.

En cada paso toma una acción aleatoria.

Suma las recompensas para saber el puntaje total del episodio.

Devuelve una lista con los puntajes obtenidos en cada episodio.

In [14]:
def evaluate_random(env, episodes=5, save_video_path=None):
    """
    Ejecuta el entorno usando acciones aleatorias y opcionalmente guarda un video.

    Parámetros:
    - env: entorno Gym/VizDoom
    - episodes: número de episodios a ejecutar
    - save_video_path: ruta para guardar video (mp4). Si es None → no se guarda.

    Devuelve:
    - lista con las recompensas totales obtenidas por episodio
    """
    scores = []         # Lista donde guardamos la recompensa total por episodio
    writer = None       # Objeto de VideoWriter (se crea solo si es necesario)

    # -----------------------------------------------------------
    # Crear el VideoWriter si el usuario quiere guardar video
    # -----------------------------------------------------------
    if save_video_path:
        writer = cv2.VideoWriter(
            save_video_path,
            cv2.VideoWriter_fourcc(*"mp4v"),  # Codec MP4
            20,                                # FPS del video de salida
            (320, 240)                         # Tamaño del video (W, H)
        )

    # -----------------------------------------------------------
    # Jugar episodios con política aleatoria
    # -----------------------------------------------------------
    for ep in range(episodes):
        obs = env.reset()           # Reinicia el entorno
        frame = get_frame(obs)      # Procesa el primer frame
        done = False                # Bandera de fin de episodio
        ep_reward = 0               # Acumulador de recompensa por episodio

        # -------------------------------------------------------
        # Ejecutar hasta que el episodio termine
        # -------------------------------------------------------
        while not done:

            # Acción tomada al azar (política completamente aleatoria)
            action = env.action_space.sample()

            # Ejecutar acción en el entorno
            obs, reward, done, info = env.step(action)

            # Acumular recompensa obtenida
            ep_reward += reward

            # Procesar el frame actual
            frame = get_frame(obs)

            # Si estamos grabando video, escribir frame
            if writer:
                writer.write(to_bgr(frame))  # Convertimos gris→BGR para OpenCV

        # Guardamos el puntaje total del episodio
        scores.append(ep_reward)

    # -----------------------------------------------------------
    # Cerrar writer del video al terminar
    # -----------------------------------------------------------
    if writer:
        writer.release()

    # Devuelve la lista de recompensas de todos los episodios
    return scores


Esta función prueba tu agente entrenado, usando su red neuronal para decidir las acciones.

Qué hace:

Reinicia el entorno.

Preprocesa y apila 4 frames (como en entrenamiento).

El agente elige la mejor acción (argmax).

Suma la recompensa total del episodio.

Guarda los frames como video.

Devuelve los puntajes obtenidos por el agente.

In [15]:
def evaluate_agent(env, agent, episodes=5, device="cpu", save_video_path=None):
    """
    Evalúa un agente entrenado ejecutando varios episodios.

    - Usa el modelo `agent` para seleccionar acciones.
    - Maneja un stack de 4 frames como entrada de la red.
    - Puede guardar el episodio en un video MP4.
    - Devuelve las recompensas totales por episodio.
    """

    scores = []        # Lista donde se almacenan las recompensas finales
    writer = None      # VideoWriter, si se usa

    # -----------------------------------------------------------
    # Configurar el VideoWriter si se pidió guardar video
    # -----------------------------------------------------------
    if save_video_path:
        writer = cv2.VideoWriter(
            save_video_path,
            cv2.VideoWriter_fourcc(*"mp4v"),  # Codec de video
            20,                               # FPS del video
            (320, 240)                        # Resolución (ancho, alto)
        )

    # -----------------------------------------------------------
    # Recorrer cada episodio
    # -----------------------------------------------------------
    for ep in range(episodes):

        obs = env.reset()      # Reinicia el entorno
        frame = get_frame(obs) # Obtiene un frame ya normalizado a gris

        done = False
        total_reward = 0

        # -------------------------------------------------------
        # Crear el frame stack inicial (4 frames iguales)
        # Cada frame procesado tiene shape (1, 84, 84)
        # -------------------------------------------------------
        f = preprocess(frame)
        frame_stack = [f for _ in range(4)]

        # -------------------------------------------------------
        # Ejecutar el episodio hasta que done == True
        # -------------------------------------------------------
        while not done:

            # Unir los 4 frames en un único tensor de entrada
            # shape final: (1, 4, 84, 84)
            stacked = torch.cat(frame_stack, dim=0).unsqueeze(0).to(device)

            # Forward sin gradientes
            with torch.no_grad():
                logits, value = agent(stacked)

            # Convertir logits a probabilidades
            probs = torch.softmax(logits, dim=1)

            # Muestreo estocástico de la acción
            action = torch.multinomial(probs, 1).item()

            # Ejecutar acción en el entorno
            obs, reward, done, info = env.step(action)
            total_reward += reward

            # Procesar el nuevo frame
            frame = get_frame(obs)
            new_f = preprocess(frame)

            # Actualizar frame stack: eliminar el más viejo, agregar el nuevo
            frame_stack.pop(0)
            frame_stack.append(new_f)

            # Escribir frame en video (conversión a 3 canales)
            if writer:
                writer.write(to_bgr(frame))

        # Guardar recompensa total del episodio
        scores.append(total_reward)

    # -----------------------------------------------------------
    # Cerrar el archivo de video
    # -----------------------------------------------------------
    if writer:
        writer.release()

    return scores


Selecciona GPU o CPU

Usa CUDA si está disponible para acelerar el entrenamiento.

Carga el escenario de Doom

Abre el archivo basic.cfg para crear el entorno del juego.

Crea el agente

CNNBase: red neuronal que verá 4 frames (stack) y escogerá acciones.

El número de acciones viene del entorno.

Crea el algoritmo PPO

PPO usa la red del agente para aprender a jugar Doom.

Entrena el agente

Corre 500.000 pasos de entrenamiento.

Cada 1024 pasos junta experiencias y actualiza la red.

Guarda el modelo en doom_agent.pth.

In [16]:
# ===============================
# CONFIGURACIÓN DEL DISPOSITIVO
# ===============================
# Selecciona GPU si está disponible; de lo contrario, usa CPU.
device = "cuda" if torch.cuda.is_available() else "cpu"


# ===============================
# CARGA DEL ENTORNO VIZDOOM
# ===============================
# Ruta del archivo .cfg que define el escenario del juego.
# IMPORTANTE: Ajustar la ruta al archivo de tu escenario.
config_path = "/kaggle/working/ViZDoom/scenarios/basic.cfg"

# Crear el entorno de entrenamiento basado en VizDoom.
env = VizdoomGymEnv(config_path)


# ===============================
# DEFINICIÓN DEL AGENTE
# ===============================
# CNNBase:
#   - input_channels=4 → usa un stack de 4 frames consecutivos como entrada.
#   - n_actions → número de acciones permitidas por el entorno.
agent = CNNBase(input_channels=4, n_actions=env.action_space.n).to(device)


# ===============================
# CONFIGURAR PPO (Proximal Policy Optimization)
# ===============================
# Conecta el agente con el algoritmo PPO, usando el dispositivo elegido.
ppo = PPO(agent, device=device)


# ===============================
# ENTRENAMIENTO DEL AGENTE
# ===============================
# train():
#   - total_timesteps → número total de pasos de entrenamiento.
#   - rollout_length → tamaño de cada batch de experiencias antes de actualizar PPO.
#   - save_path → nombre del archivo donde se guardará el modelo entrenado.
train(
    env,               # Entorno del juego
    agent,             # Agente con su red neuronal convolucional
    ppo,               # Algoritmo PPO
    total_timesteps=500_000,  # Pasos totales de entrenamiento
    rollout_length=1024,      # Longitud de cada rollout
    device=device,            # CPU o GPU
    save_path="doom_agent.pth"  # Archivo final del modelo
)


Timestep 1024 — Episode 17 — Guardado doom_agent.pth
Timestep 2048 — Episode 40 — Guardado doom_agent.pth
Timestep 3072 — Episode 54 — Guardado doom_agent.pth
Timestep 4096 — Episode 67 — Guardado doom_agent.pth
Timestep 5120 — Episode 81 — Guardado doom_agent.pth
Timestep 6144 — Episode 95 — Guardado doom_agent.pth
Timestep 7168 — Episode 108 — Guardado doom_agent.pth
Timestep 8192 — Episode 122 — Guardado doom_agent.pth
Timestep 9216 — Episode 136 — Guardado doom_agent.pth
Timestep 10240 — Episode 149 — Guardado doom_agent.pth
Timestep 11264 — Episode 163 — Guardado doom_agent.pth
Timestep 12288 — Episode 177 — Guardado doom_agent.pth
Timestep 13312 — Episode 190 — Guardado doom_agent.pth
Timestep 14336 — Episode 204 — Guardado doom_agent.pth
Timestep 15360 — Episode 218 — Guardado doom_agent.pth
Timestep 16384 — Episode 231 — Guardado doom_agent.pth
Timestep 17408 — Episode 245 — Guardado doom_agent.pth
Timestep 18432 — Episode 258 — Guardado doom_agent.pth
Timestep 19456 — Episode 

In [17]:
# ===============================
# EVALUACIÓN DE AGENTES
# ===============================

# 1. Evaluar un agente aleatorio (benchmark inicial)
# --------------------------------------------------
# evaluate_random():
#   - Ejecuta el entorno sin un modelo entrenado.
#   - Elige acciones al azar para servir como referencia.
#   - episodes=5 → realiza 5 episodios completos.
#   - save_video_path → genera un video mostrando el agente actuando al azar.
scores_random = evaluate_random(
    env,
    episodes=5,
    save_video_path="random_agent.mp4"
)


# 2. Evaluar el agente entrenado con PPO
# ---------------------------------------
# evaluate_agent():
#   - Usa el modelo entrenado (agent) para interactuar con el entorno.
#   - device → asegura que el modelo corre en CPU o GPU según corresponda.
#   - save_video_path → guarda un video con el desempeño del agente entrenado.
scores_trained = evaluate_agent(
    env,
    agent,
    episodes=5,
    device=device,
    save_video_path="trained_agent.mp4"
)


# ===============================
# IMPRESIÓN DE RESULTADOS
# ===============================
# Mostramos los puntajes de ambos agentes para comparar el rendimiento.
print("Random scores:", scores_random)
print("Trained scores:", scores_trained)


# ===============================
# CIERRE DEL ENTORNO
# ===============================
# Siempre se debe cerrar el entorno al terminar para liberar memoria.
env.close()


Random scores: [-196.0, 54.0, -325.0, 75.0, 21.0]
Trained scores: [95.0, -315.0, -325.0, -315.0, -325.0]
